# Factory Method (metoda wytwórcza)

# Ćwiczenie: Shapes

### Kod początkowy

In [65]:
class Shape(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class Circle(Shape):
    def __init__(self, x, y, r):
        # super(Circle, self).__init__(x, y)  # Python 2
        super().__init__(x, y)  # Python 3
        self.r = r
        
    def __repr__(self):
        return 'Circle x={} y={} r={}'.format(self.x, self.y, self.r)
        
class Rectangle(Shape):
    def __init__(self, x, y, w, h):
        # super(Rectangle, self).__init__(x, y)  # Python 2
        super().__init__(x, y)  # Python 3
        self.w = w
        self.h = h
        
    def __repr__(self):
        return 'Rectangle x={} y={} w={} h={}'.format(self.x, self.y, self.w, self.h)

In [66]:
class Drawing(object):
    def __init__(self, shapes):
        self._shapes = shapes
        
    def __repr__(self):
        return '<Drawing {}>'.format(str(self._shapes))
        
    @classmethod
    def from_string(cls, string):
        shapes = []
        for line in string.split('\n'):
            line = line.strip()
            if not line:
                continue
            ...
            shapes.append(...)
        return cls(shapes)

In [67]:
raw_shapes = '''
Circle 15 10 14
Rectangle 30 30 100 150
Circle 40 20 5
Square 30 100 20
'''

### Rozwiązanie bez Factory Method

In [68]:
class BadDrawing(object):
    def __init__(self, shapes):
        self._shapes = shapes
        
    def __repr__(self):
        return '<BadDrawing {}>'.format(str(self._shapes))
        
    @classmethod
    def from_string(cls, string):
        shapes = []
        for line in string.split('\n'):
            line = line.strip()
            if not line:
                continue
            shape_name, *parameters = line.split()
            parameters = map(int, parameters)
            if shape_name == 'Circle':
                shape = Circle(*parameters)
            elif shape_name == 'Rectangle':
                shape = Rectangle(*parameters)
            elif shape_name == 'Square':
                x, y, a = parameters
                shape = Rectangle(x, y, a, a)
            else:
                raise TypeError
            shapes.append(shape)
        return cls(shapes)

In [69]:
BadDrawing.from_string(raw_shapes)

<BadDrawing [Circle x=15 y=10 r=14, Rectangle x=30 y=30 w=100 h=150, Circle x=40 y=20 r=5, Rectangle x=30 y=100 w=20 h=20]>

### Rozwiązanie z Factory Method

In [70]:
shape_factory = {
    'Circle': Circle,
    'Rectangle': Rectangle,
    'Square': lambda x, y, a: Rectangle(x, y, a, a),
}

In [71]:
class Drawing(object):
    def __init__(self, shapes):
        self._shapes = shapes
        
    def __repr__(self):
        return '<Drawing {}>'.format(str(self._shapes))
        
    @classmethod
    def from_string(cls, string, shape_factory):
        shapes = []
        for line in string.split('\n'):
            line = line.strip()
            if not line:
                continue
            shape_name, *parameters = line.split()
            parameters = map(int, parameters)
            try:
                shape_creator = shape_factory[shape_name]
            except KeyError:
                raise TypeError
            else:
                shape = shape_creator(*parameters)
            shapes.append(shape)
        return cls(shapes)

In [72]:
Drawing.from_string(raw_shapes, shape_factory)

<Drawing [Circle x=15 y=10 r=14, Rectangle x=30 y=30 w=100 h=150, Circle x=40 y=20 r=5, Rectangle x=30 y=100 w=20 h=20]>

### Rozwiązanie z Factory Method i użyciem `Shape.__subclasses__`

In [73]:
class Shape(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @classmethod
    def create_factory(cls):
        shape_factory = {}
        for subclass in cls.__subclasses__():
            shape_factory[subclass.__name__] = subclass
            # recurse if you want to include subsubclasses and so on
            # shape_factory.update(subclass.create_factory())
        return shape_factory
        
class Circle(Shape):
    def __init__(self, x, y, r):
        super(Circle, self).__init__(x, y)
        self.r = r
        
    def __repr__(self):
        return 'Circle x={} y={} r={}'.format(self.x, self.y, self.r)
        
class Rectangle(Shape):
    def __init__(self, x, y, w, h):
        super(Rectangle, self).__init__(x, y)
        self.w = w
        self.h = h
    def __repr__(self):
        return 'Rectangle x={} y={} w={} h={}'.format(self.x, self.y, self.w, self.h)
    
class Square(Shape):
    def __new__(cls, x, y, a):
        return Rectangle(x, y, a, a)

In [74]:
Shape.create_factory()

{'Circle': __main__.Circle,
 'Rectangle': __main__.Rectangle,
 'Square': __main__.Square}

In [75]:
class Drawing(object):
    def __init__(self, shapes):
        self._shapes = shapes
        
    def __repr__(self):
        return '<Drawing {}>'.format(str(self._shapes))
        
    @classmethod
    def from_string(cls, string):
        shape_factory = Shape.create_factory()
        shapes = []
        for line in string.split('\n'):
            line = line.strip()
            if not line:
                continue
            shape_name, *parameters = line.split()
            parameters = map(int, parameters)
            try:
                shape_creator = shape_factory[shape_name]
            except KeyError:
                raise TypeError
            else:
                shape = shape_creator(*parameters)
            shapes.append(shape)
        return cls(shapes)

In [78]:
Drawing.from_string(raw_shapes)

<Drawing [Circle x=15 y=10 r=14, Rectangle x=30 y=30 w=100 h=150, Circle x=40 y=20 r=5, Rectangle x=30 y=100 w=20 h=20]>